# Configuration

Upon first import, `osyris` will create a configuration file located in `/home/user/.osyris/config_osyris.py`.
This will then be loaded by `osyris`, and allows easy configuration for the default colormap, spatial scale, etc.

## Parameters

The `parameters` dictionary inside the configuration file contains the following default values:

- `"scale"`: the spatial scale to use for distances and cell sizes
- `"path"`: the directory where data outputs are stored (this can be an absolute or relative path)
- `"select"`: a cell selection criterion for selective loading
- `"cmap"`: the default Matplotlib colormap to use
- `"render_mode"`: the default rendering mode for figures (e.g. `"image"`, `"contour"`, `"contourf"`...)
- `"sortby"`: a dict of keys (e.g. `"identity"`) to be used for sorting data groups (`"part"`, `"hydro"`...) upon load. Set to `None` to disable sorting.

## Additional units

This function creates additional units that you use often, and do not wish to re-defined all the time.
The syntax is the following:

In [ ]:
def additional_units(ureg):
    ureg.define('solar_mass = 1.9889e+33 * g = M_sun = M_sol')
    ureg.define('radiation_constant = 7.5659146e-015 * erg / cm^3 / K^4 = ar')

## Additional variables

It is often useful to define some additional variables that are to be computed every time data is loaded.
These commonly include cell-centered magnetic field, or temperature.

It is recommended to place your variables in a `try/except` block,
which will prevent errors if the variables are not found,
for instance when loading data from a different simulation.

For instance

In [ ]:
def additional_variables(data):
    # Magnetic field
    try:
        data["hydro"]["B_field"] = 0.5 * (data["hydro"]["B_left"] + data["hydro"]["B_right"])
    except KeyError:
        pass

    # Mass
    try:
        data["hydro"]["mass"] = (data["hydro"]["density"] * data["amr"]["dx"]**3).to("M_sun")
    except KeyError:
        pass